In [1]:
import feedparser
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import os
import pymongo

In [2]:
# Get the mongodb password from an environment variable
mongoPass = os.environ['mongoPass']

# Establish the remote connection to the mongo data base
myclient = pymongo.MongoClient("mongodb+srv://axme100:{}@cluster0-5jopz.mongodb.net/test?retryWrites=true&w=majority".format(mongoPass))

# This is the name of the cluster stored on mongo atlas
mydb = myclient["finalProject"]

# Create a new colection called raw article 
mycol = mydb["rawArticles"]

KeyError: 'mongoPass'

In [16]:
# Define the class that will be created
class rawArticle:
    def __init__(self, url, title, date, publication, articleText):
        self.url = url
        self.title = title
        self.date = date
        self.publication = publication
        self.articleText = articleText
        
    def saveToDatabase(self):
        # Save the entry into the mongo database
        mycol.insert_one({'url': self.url,
                         'title': self.title,
                         'date': self.date,
                         'publication': self.publication,
                         'articleText': self.articleText})

In [17]:
# Define a class of scraper
class scraper:
    
    def scrape_jornada(self):
        # Hard coded URL to be scraped
        url="https://www.jornada.com.mx/rss/edicion.xml?v=1"
        
        # Parse the XML feed
        feedBurner = feedparser.parse(url)
        
        # Loop through all of the entries
        for entry in feedBurner['entries']:
    
            # Get the page that we want to scrape
            html = requests.get(entry['link'])
        
            # Save the page as a beautiful soup object
            bs = BeautifulSoup(html.text, 'html.parser')
    
            articleText = bs.find("div", {"id": "article-text"}).get_text().replace('\n','').strip()
    
            # Create an article object
            myArticle = rawArticle(url=entry['link'],
                          title=entry['title'],
                          date=entry['published'],
                          publication="La Jornada",
                          articleText=articleText)
        
            # Call the object's method to save to a mongo data base
            myArticle.saveToDatabase()
    
    def scrape_la_presna():
        
        # Get the soup of the page where the sub rss links are posted
        url='http://laprensa.mx/rss.asp'
        html = requests.get(url)
        bs = BeautifulSoup(html.text, 'html.parser')
        
        # Get a list of all the links that are part of the sub rss feed
        rssSections = bs.find_all("div", {"class": "secc"})
        rssSubLinks = []
        for section in rssSections:
            link = section.find('a', href=True)
            rssSubLinks.append(link['href'])
        
        
        # Iterate over and parse each of the links in the rssSubLinks
        for link in rssSubLinks:
            feedBurner = feedparser.parse(link)
    
            # Iterate over and parse each of the entries in the rss page
            for entry in feedBurner['entries']:
                
                # Get the soup page of to scrape
                html = requests.get(entry['link'])
                bs = BeautifulSoup(html.text, 'html.parser')

                # It appears that all of the text is located in a div caled text
                textDiv = bs.find('div', {'id': 'texto'})

                # Get rid of all the divs that are within this div because
                # some pages have tables and images with text that appear to be wrapped in divs
                for div in textDiv("div"):
                    div.decompose()
    
                # The result of the previous blocks, 
                articleText = textDiv.get_text().replace('\n','').strip()
                
                # Create an article object
                myArticle = rawArticle(url=entry['link'],
                          title=entry['title'],
                          date=entry['published'],
                          publication="La Jornada",
                          articleText=articleText)
        
                # Call the object's method to save to a mongo data base
                myArticle.saveToDatabase()

In [18]:
myScrapper = scraper()
myScrapper.scrape_jornada()
myScrapper.scarpe_la_prensa()

AttributeError: 'NoneType' object has no attribute 'get_text'